In [ ]:
import os
from connection_complexity.data.raw_data.EDF.edf_helpers import read_edf
import connection_complexity.data.rereference_helpers as reref_helpers
from tqdm.notebook import tqdm

from mne import make_fixed_length_epochs
import pandas as pd
import numpy as np

In [4]:
project_path = "/home/dan/data/connectivity/pyspi_testing"

In [5]:
# set random seed
np.random.seed(42)

# set random epoch indices
random_epoch_idx = np.random.randint(0,500+1,size=2)
random_epoch_idx


array([102, 435])

In [6]:
pids = []

for file in os.listdir("/media/dan/Data/data/baseline_patients/baseline_edfs"):
    if file.endswith(".EDF"):
        pids.append(file.split("_")[0])

pids = list(set(pids))
pids = sorted([int(pid) for pid in pids])

In [ ]:
100.102.168.79

In [7]:
bad_channels_path = "/media/dan/Big/manuiscript_0001_hfo_rates/data/bad_ch_review.xlsx"


bad_channels = pd.read_excel(bad_channels_path)
bad_channels["ch"] = bad_channels["ch"].apply(lambda x: x.replace(" ", ""))
bad_channels["ch"] = bad_channels["ch"].apply(lambda x: x.replace("-", "~"))
bad_channels["use"] = bad_channels["use"].fillna(1)
bad_channels["use2"] = bad_channels["use2"].fillna(1)
bad_channels["use"] = bad_channels["use"].astype(bool)
bad_channels["use2"] = bad_channels["use2"].astype(bool)

# OR bad_channel columns
bad_channels["bad_channel"] = ~(bad_channels["use"] & bad_channels["use2"])

In [10]:
npy_output_path = f"/media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/run_data"

pid = 14
file = f"/media/dan/Data/data/baseline_patients/baseline_edfs/{pid:03}_Baseline.EDF"
print(file)
raw = read_edf(file, preload=True)

# drop bad channels
try:
    raw = raw.drop_channels(bad_channels[(bad_channels["bad_channel"] == 1) & (bad_channels["pid"] == pid)]["ch"].values)
except:
    bc = bad_channels[(bad_channels["bad_channel"] == 1) & (bad_channels["pid"] == pid)]["ch"]
    for b in bc:
        try:
            raw = raw.drop_channels(b)
        except:
            pass

# # filter
sfreq = raw.info["sfreq"]
# nyq_freq = raw.info["sfreq"] // 2 - 1
# line_freq = 60
# l_freq = 0.5
# h_freq = min(nyq_freq, 300)
# raw = raw.filter(l_freq=l_freq, h_freq=h_freq)
# freqs = np.arange(line_freq, max(h_freq, nyq_freq), line_freq)
# raw = raw.notch_filter(freqs=freqs, method="fir")

raw = raw.set_eeg_reference(ref_channels="average", projection=False, verbose=False)

# downsample raw to 256 Hz
target_sfreq = 256
if sfreq != target_sfreq:
    raw = raw.resample(target_sfreq, npad="auto")
    sfreq = raw.info["sfreq"]

# non overlapping epochs
dur_sec = .5
overlap_sec = 0
epochs = make_fixed_length_epochs(raw, duration=dur_sec, overlap=overlap_sec, preload=True)
epochs = epochs.get_data()

for i, e in zip(random_epoch_idx, epochs): 
    start_idx = int(sfreq * dur_sec) * i
    end_idx = start_idx + int(sfreq * dur_sec)
    np.save(os.path.join(npy_output_path, f"{pid:03}_epoch_{start_idx:010d}-{end_idx:010d}.npy"), e[:3, :])
    break

/media/dan/Data/data/baseline_patients/baseline_edfs/014_Baseline.EDF


In [ ]:
e[:3,:].shape

(3, 1024)

: 

In [ ]:
npy_output_path = f"/media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/run_data/npy"
contacts_used_output_path = f"/media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/run_data/contacts_used"
# make output paths
if not os.path.exists(npy_output_path):
    os.makedirs(npy_output_path)

if not os.path.exists(contacts_used_output_path):
    os.makedirs(contacts_used_output_path)

for pid in tqdm(pids):
    file = f"/media/dan/Data/data/baseline_patients/baseline_edfs/{pid:03}_Baseline.EDF"
    print(file)
    raw = read_edf(file, preload=True)

    # drop bad channels
    try:
        raw = raw.drop_channels(bad_channels[(bad_channels["bad_channel"] == 1) & (bad_channels["pid"] == pid)]["ch"].values)
    except:
        bc = bad_channels[(bad_channels["bad_channel"] == 1) & (bad_channels["pid"] == pid)]["ch"]
        for b in bc:
            try:
                raw = raw.drop_channels(b)
            except:
                pass

    # filter
    sfreq = raw.info["sfreq"]
    nyq_freq = raw.info["sfreq"] // 2 - 1
    line_freq = 60
    l_freq = 0.5
    h_freq = min(nyq_freq, 300)
    raw = raw.filter(l_freq=l_freq, h_freq=h_freq)
    freqs = np.arange(line_freq, max(h_freq, nyq_freq), line_freq)
    raw = raw.notch_filter(freqs=freqs, method="fir")

    raw = raw.set_eeg_reference(ref_channels="average", projection=False, verbose=False)

    data = raw.get_data()

    contacts_used = raw.ch_names
    # save csv of contacts used
    pd.DataFrame(contacts_used, columns=["contact"]).to_csv(os.path.join(contacts_used_output_path, f"{pid:03}_contacts_used.csv"), index=False)
    
    # non overlapping epochs
    dur_sec = .5
    overlap_sec = 0
    epochs = make_fixed_length_epochs(raw, duration=dur_sec,overlap=overlap_sec, preload=True)
    epochs = epochs.get_data()

    for i, e in zip(random_epoch_idx, epochs): 
        start_idx = int(sfreq * dur_sec) * i
        end_idx = start_idx + int(sfreq * dur_sec)
        np.save(os.path.join(npy_output_path, f"{pid:03}_epoch_{start_idx:010d}-{end_idx:010d}.npy"), e)


  0%|          | 0/72 [00:00<?, ?it/s]

/media/dan/Data/data/baseline_patients/baseline_edfs/001_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/004_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/006_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/008_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/009_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/010_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/011_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/012_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/013_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/014_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/016_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/017_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/018_Baseline.EDF
/media/dan/Data/data/baseline_patients/baseline_edfs/019_Baseline.EDF
/media/dan/Data/data

In [35]:
# define the YAML file location
yaml_file = os.path.join(npy_output_path,"run_config.yaml")

# remove the file if it already exists
if os.path.exists(yaml_file):
    os.remove(yaml_file)

# ps -> rows = processes; columns = time pts.
dim_order = "ps"


import glob
files = list(sorted(list(glob.glob(os.path.join(npy_output_path, "*.npy")))))

for f in sorted(files): # start 1 minute in only use first 5 seconds
    # get the epoch number
    i = os.path.basename(f).split("_")[-1].split(".")[0]

    pid = os.path.basename(f).split("_")[0]
    # define template string and fill in variables
    
    lbl = f"{pid}_epoch_{i}"
    path = os.path.join(npy_output_path, lbl)
    yaml_string = "{{file: {file}.npy, name: {key}, dim_order: {dim_order}, labels: [{key}]}}\n"
    yaml_string_formatted = f"- {yaml_string.format(file=path, key=lbl, dim_order=dim_order)}"

    # append line to file
    with open(yaml_file, "a") as f:
        f.write(yaml_string_formatted)

# make sure yaml loads

In [36]:
import yaml
with open(yaml_file) as d:
    yf = yaml.load(d,Loader=yaml.FullLoader)